In [1]:
# Get the data for expeirment
import pandas as pd 
import numpy as np
import time
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, RBF
import seaborn as sns
import TsModel
import GprModel
import TsModel_constant
import torch
from sklearn.metrics import r2_score, root_mean_squared_error
import matplotlib.pyplot as plt
import os
os.environ["KERAS_BACKEND"] = "torch"
import keras
from keras import layers, models

device = torch.device("cuda:0")
print(f"Keras version is {keras.__version__}")
print(f"Num GPUs Available: {torch.cuda.device_count()}")
plt.rcParams.update({'font.size': 20})

Keras version is 3.1.1
Num GPUs Available: 1


In [16]:
exp30_2_results = {
    'gp_tsl_train_accuracy':gp_tsl_train_accuracy,
    'gp_tsl_test_accuracy':gp_tsl_test_accuracy,
    'gp_tsl_r2s':gp_tsl_r2s,
    'gp_tsl_time':gp_tsl_time,
    'gp_tsc_train_accuracy':gp_tsc_train_accuracy,
    'gp_tsc_test_accuracy':gp_tsc_test_accuracy,
    'gp_tsc_r2s':gp_tsc_r2s,
    'gp_tsc_time':gp_tsc_time,
    'gp_nn_tsl_train_accuracy':gp_nn_tsl_train_accuracy,
    'gp_nn_tsl_test_accuracy':gp_nn_tsl_test_accuracy,
    'gp_nn_tsl_r2s':gp_nn_tsl_r2s,
    'nn_tsl_r2s':nn_tsl_r2s,
    'gp_nn_tsl_time':gp_nn_tsl_time,
}

In [17]:
#%store exp30_2_results 

Stored 'exp30_2_results' (dict)


In [2]:
# Train and test the solar data set
x_train = pd.read_csv('Data/FES/small_wind_turbine_x_train.csv')
x_test = pd.read_csv('Data/FES/small_wind_turbine_x_test.csv')
y_train = pd.read_csv('Data/FES/small_wind_turbine_y_train.csv')
y_test = pd.read_csv('Data/FES/small_wind_turbine_y_test.csv')

In [3]:
number_of_folds = 10

# Source: From the TSmodel
def preprocess_data(train_x, test_x, train_y, test_y):
    feature_scaler = StandardScaler()
    scaled_train_x = feature_scaler.fit_transform(train_x)
    scaled_test_x = feature_scaler.transform(test_x)

    # target_scaler = StandardScaler()  
    # scaled_train_y = target_scaler.fit_transform(train_y)
    # scaled_test_y = target_scaler.transform(test_y)    
    # return scaled_train_x, scaled_test_x, scaled_train_y, scaled_test_y, feature_scaler, target_scaler    

    return scaled_train_x, scaled_test_x, feature_scaler    

def build_neural_network_model():
    model = models.Sequential()
    model.add(layers.Dense(9, activation=keras.activations.tanh))    
    model.add(layers.Dense(18, activation=keras.activations.tanh))    
    model.add(layers.Dense(1, activation=keras.activations.relu))
    return model

In [4]:
x_train, x_test, feature_scaler = preprocess_data(x_train, x_test, y_train, y_test)
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [13]:
def do_nn_exp():
    neural_network_model = build_neural_network_model()

    neural_network_model.compile(
        optimizer=keras.optimizers.Adam(),
        loss=keras.losses.MeanSquaredError(),
        metrics=[keras.metrics.RootMeanSquaredError()]
    )

    start_time = time.time()
    history = neural_network_model.fit(
        x_train, 
        y_train,
        epochs=100, 
        shuffle=True
    )
    time_used = time.time() - start_time
    
    return neural_network_model

In [14]:
nn_model = do_nn_exp()

Epoch 1/100
4901/4901 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - loss: 14992.1475 - root_mean_squared_error: 122.3602
Epoch 2/100
4901/4901 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - loss: 10148.2480 - root_mean_squared_error: 100.6803
Epoch 3/100
4901/4901 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - loss: 7406.2417 - root_mean_squared_error: 86.0070
Epoch 4/100
4901/4901 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - loss: 5381.4761 - root_mean_squared_error: 73.2978
Epoch 5/100
4901/4901 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - loss: 4596.8691 - root_mean_squared_error: 67.6430
Epoch 6/100
4901/4901 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - loss: 3550.2556 - root_mean_squared_error: 59.5529
Epoch 7/100
4901/4901 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - loss: 2976.3826 - root_mean_squared_error: 54.5053
Epoch 8/100
4901/4901 ━━━━━━━━━━━━━━━━━━━━ 19s 4ms/step - loss: 2482.2488 - root_mean_squared_error: 49.7558
Epoch 9/100
4901/4901 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - loss: 2050.3608 - root_mean_squared_error: 45.1443
Epoch 10/100
49

In [10]:
# TS model
# Linear conclusion

def ts_relu_wrapper(pred):
    return np.maximum(0, pred)

# TS with linear conclusion
tsl_model = TsModel.TsModel(number_of_rules=25, fuzzification_coefficient=1.2)

start_time = time.time()
tsl_model.fit(x_train, y_train)
time_used = time.time() - start_time

FCM training RMSE: 18.009862959525147


In [8]:
# TS model
# Constant conclusion


tsc_model = TsModel_constant.TsModel_constant(number_of_rules=25, fuzzification_coefficient=1.2)

start_time = time.time()
tsc_model.fit(x_train, y_train)
time_used = time.time() - start_time

(156830, 250)
[[1.00291236e-05 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [6.18903397e-02 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.35838705e-06 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [6.49188947e-05 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [5.11619604e-05 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.29403555e-06 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
(156830, 25)
[[1.00291236e-05 1.06301311e-01 7.18777331e-06 ... 2.69185954e-03
  4.80988098e-06 9.20827024e-06]
 [6.18903397e-02 2.16325127e-10 9.89626327e-05 ... 4.49841434e-09
  3.45561850e-07 5.77556323e-05]
 [3.35838705e-06 6.99867117e-01 4.45664667e-06 ... 1.72199648e-03
  3.62516114e-06 6.14252794e-07]
 ...
 [6.49188947e-05 1.53340762e-07 3.91083108e-04 ... 2.76815403e-06
  1.1

In [8]:
def get_tsl_model(x_train, y_train):
    tsl_model = TsModel.TsModel(number_of_rules=25, fuzzification_coefficient=1.2)
    tsl_model.fit(x_train, y_train)
    return tsl_model

def get_tsc_model(x_train, y_train):
    tsc_model = TsModel_constant.TsModel_constant(number_of_rules=25, fuzzification_coefficient=1.2)
    tsc_model.fit(x_train, y_train)
    return tsc_model

In [5]:
# GPR model
gp_train_accuracy = []
gp_test_accuracy = []
gp_r2s = []
gp_time = []
gpr_model = None

for fold in range(number_of_folds):
    
    kernel = 1.0 * Matern(length_scale=1.0, nu=1.5)
    gpr_model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9, normalize_y=False)
    indices = np.random.choice(x_train.shape[0], 2000, replace=False)      

    start_time = time.time()
    gpr_model.fit(x_train[indices,:], y_train[indices,:])
    time_used = time.time() - start_time

    train_means = gpr_model.predict(x_train)
    test_means = gpr_model.predict(x_test)

    train_rmse = root_mean_squared_error(y_train, train_means)
    test_rmse = root_mean_squared_error(y_test, test_means)

    gp_train_accuracy.append(train_rmse)
    gp_test_accuracy.append(test_rmse)
    gp_r2 = r2_score(y_test, test_means)
    gp_r2s.append(gp_r2)
    gp_time.append(time_used)

gp_train_accuracy_mean = np.round(np.mean(gp_train_accuracy), 3)
gp_train_accuracy_std = np.round(np.std(gp_train_accuracy), 3)
gp_test_accuracy_mean = np.round(np.mean(gp_test_accuracy), 3)
gp_test_accuracy_std = np.round(np.std(gp_test_accuracy),3)
gp_r2_mean = np.round(np.mean(gp_r2s),3)
gp_r2_std = np.round(np.std(gp_r2s),3)
gp_time_mean = np.round(np.mean(gp_time),3)
gp_time_std = np.round(np.std(gp_time),3)

print(f"GP Train RMSE: {gp_train_accuracy_mean} ± {gp_train_accuracy_std}")
print(f"GP Test RMSE: {gp_test_accuracy_mean} ± {gp_test_accuracy_std}")
print(f"GP R2: {gp_r2_mean} ± {gp_r2_std}")
print(f"GP Time: {gp_time_mean} ± {gp_time_std}")


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\_gpr.py:663: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\_gpr.py:663: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.

GP Train RMSE: 67.979 ± 52.825
GP Test RMSE: 69.018 ± 53.051
GP R2: 0.496 ± 0.534
GP Time: 104.475 ± 34.704


In [11]:
# GPR with TSL
gp_tsl_train_accuracy = []
gp_tsl_test_accuracy = []
gp_tsl_r2s = []
gp_tsl_time = []
gpr_tsl_model = None

# From ts linear
for fold in range(number_of_folds):
    
    kernel = 1.0 * Matern(length_scale=1.0, nu=1.5)
    gpr_tsl_model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9, normalize_y=False)

    tsl_model = get_tsl_model(x_train, y_train)
    start_time = time.time()
    gpr_tsl_model.fit(tsl_model.cen, ts_relu_wrapper(tsl_model.predict(tsl_model.cen + 1e-8)))
    time_used = time.time() - start_time

    train_means = gpr_tsl_model.predict(x_train)
    test_means = gpr_tsl_model.predict(x_test)

    train_rmse = root_mean_squared_error(y_train, train_means)
    test_rmse = root_mean_squared_error(y_test, test_means)

    gp_tsl_train_accuracy.append(train_rmse)
    gp_tsl_test_accuracy.append(test_rmse)
    gp_r2 = r2_score(y_test, test_means)
    gp_tsl_r2s.append(gp_r2)
    gp_tsl_time.append(time_used)

gp_train_accuracy_mean = np.round(np.mean(gp_tsl_train_accuracy), 3)
gp_train_accuracy_std = np.round(np.std(gp_tsl_train_accuracy), 3)
gp_test_accuracy_mean = np.round(np.mean(gp_tsl_test_accuracy), 3)
gp_test_accuracy_std = np.round(np.std(gp_tsl_test_accuracy),3)
gp_r2_mean = np.round(np.mean(gp_tsl_r2s),3)
gp_r2_std = np.round(np.std(gp_tsl_r2s),3)
gp_time_mean = np.round(np.mean(gp_tsl_time),3)
gp_time_std = np.round(np.std(gp_tsl_time),3)

print(f"GP Train RMSE: {gp_train_accuracy_mean} ± {gp_train_accuracy_std}")
print(f"GP Test RMSE: {gp_test_accuracy_mean} ± {gp_test_accuracy_std}")
print(f"GP R2: {gp_r2_mean} ± {gp_r2_std}")
print(f"GP Time: {gp_time_mean} ± {gp_time_std}")


FCM training RMSE: 15.656994414269844


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


FCM training RMSE: 18.260463988299175


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


FCM training RMSE: 15.919076879090703


...

FCM training RMSE: 15.799455777472495


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


FCM training RMSE: 15.855706421520086


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


GP Train RMSE: 36.159 ± 4.089
GP Test RMSE: 36.979 ± 4.244
GP R2: 0.908 ± 0.022
GP Time: 0.131 ± 0.047


In [12]:
# GPR with tsc
gp_tsc_train_accuracy = []
gp_tsc_test_accuracy = []
gp_tsc_r2s = []
gp_tsc_time = []
gpr_tsc_model = None

# From ts constant
for fold in range(number_of_folds):
    
    kernel = 1.0 * Matern(length_scale=1.0, nu=1.5)
    gpr_tsc_model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9, normalize_y=False)
    tsc_model = get_tsc_model(x_train, y_train)
    start_time = time.time()
    gpr_tsc_model.fit(tsc_model.cen, ts_relu_wrapper(tsc_model.predict(tsc_model.cen + 1e-8)))
    time_used = time.time() - start_time

    train_means = gpr_tsc_model.predict(x_train)
    test_means = gpr_tsc_model.predict(x_test)

    train_rmse = root_mean_squared_error(y_train, train_means)
    test_rmse = root_mean_squared_error(y_test, test_means)

    gp_tsc_train_accuracy.append(train_rmse)
    gp_tsc_test_accuracy.append(test_rmse)
    gp_r2 = r2_score(y_test, test_means)
    gp_tsc_r2s.append(gp_r2)
    gp_tsc_time.append(time_used)

gp_train_accuracy_mean = np.round(np.mean(gp_tsc_train_accuracy), 3)
gp_train_accuracy_std = np.round(np.std(gp_tsc_train_accuracy), 3)
gp_test_accuracy_mean = np.round(np.mean(gp_tsc_test_accuracy), 3)
gp_test_accuracy_std = np.round(np.std(gp_tsc_test_accuracy),3)
gp_r2_mean = np.round(np.mean(gp_tsc_r2s),3)
gp_r2_std = np.round(np.std(gp_tsc_r2s),3)
gp_time_mean = np.round(np.mean(gp_tsc_time),3)
gp_time_std = np.round(np.std(gp_tsc_time),3)

print(f"GP Train RMSE: {gp_train_accuracy_mean} ± {gp_train_accuracy_std}")
print(f"GP Test RMSE: {gp_test_accuracy_mean} ± {gp_test_accuracy_std}")
print(f"GP R2: {gp_r2_mean} ± {gp_r2_std}")
print(f"GP Time: {gp_time_mean} ± {gp_time_std}")


(156830, 250)
[[1.79589173e-05 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [7.98579749e-05 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.41331020e-07 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [1.14223042e-06 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [4.39778495e-08 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [8.45607749e-09 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
(156830, 25)
[[1.79589173e-05 3.73822372e-02 1.12486129e-05 ... 2.48320683e-03
  1.74317215e-05 1.40697628e-02]
 [7.98579749e-05 1.12721194e-09 2.05054346e-07 ... 1.03274156e-09
  9.99812744e-01 3.72236477e-10]
 [2.41331020e-07 1.16968190e-04 1.50204481e-06 ... 1.82371701e-04
  3.27427688e-07 1.28213810e-03]
 ...
 [1.14223042e-06 8.45791302e-08 9.69195289e-01 ... 1.49176774e-06
  1.1

e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


(156830, 250)
[[4.27889607e-01 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [5.66803205e-12 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [4.77269285e-03 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [6.34264268e-09 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.65707070e-09 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.66655965e-13 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
(156830, 25)
[[4.27889607e-01 4.53526534e-02 2.47611797e-03 ... 9.23870967e-06
  1.56926438e-05 3.27752390e-02]
 [5.66803205e-12 2.27339885e-12 1.59806783e-10 ... 9.39876212e-07
  3.46943538e-05 4.54692707e-10]
 [4.77269285e-03 8.71779025e-01 5.76103343e-04 ... 6.08115148e-07
  3.85582586e-07 2.13752361e-04]
 ...
 [6.34264268e-09 8.74793342e-09 7.20260565e-07 ... 2.21169212e-05
  7.7

e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


(156830, 250)
[[4.47693388e-06 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.54183132e-08 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.83846856e-07 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [1.40345690e-05 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [9.89832081e-01 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [9.99998800e-01 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
(156830, 25)
[[4.47693388e-06 1.07001200e-03 1.23163686e-03 ... 8.11561448e-02
  6.01408202e-06 6.45305091e-07]
 [3.54183132e-08 4.16241517e-10 1.00030251e-10 ... 3.18962308e-12
  3.98068813e-07 6.86594486e-11]
 [1.83846856e-07 1.57139729e-04 4.39941806e-02 ... 1.78218064e-03
  4.97951173e-07 1.71293072e-07]
 ...
 [1.40345690e-05 1.29418104e-06 1.07908393e-06 ... 4.26792591e-09
  5.9

...

(156830, 250)
[[2.23093828e-04 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.22529153e-06 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.00836638e-05 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [2.59954148e-06 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [6.84508791e-07 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.01232470e-08 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
(156830, 25)
[[2.23093828e-04 4.49472223e-03 1.20559486e-05 ... 1.24262365e-05
  1.21287189e-05 6.85419067e-06]
 [1.22529153e-06 2.77981935e-09 1.63865273e-02 ... 9.83285899e-01
  2.22317322e-04 2.81164763e-07]
 [1.00836638e-05 2.08580597e-03 1.39718324e-06 ... 7.83604189e-07
  4.83183977e-07 7.22491092e-07]
 ...
 [2.59954148e-06 2.31475838e-07 1.76870499e-05 ... 8.55628373e-06
  6.7

e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


(156830, 250)
[[1.22332961e-05 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [8.55300436e-05 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [6.51521526e-07 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [2.53274719e-05 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.32864951e-05 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [6.36934868e-09 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
(156830, 25)
[[1.22332961e-05 8.95622068e-06 1.05623981e-05 ... 3.96968245e-04
  2.55279762e-04 1.70312043e-05]
 [8.55300436e-05 4.12390611e-07 6.31649458e-07 ... 2.59499931e-06
  1.52074441e-07 2.65169285e-02]
 [6.51521526e-07 5.35674097e-07 2.10133458e-07 ... 4.57518450e-06
  1.45546990e-05 5.17349706e-07]
 ...
 [2.53274719e-05 5.47319849e-04 3.27672211e-06 ... 6.39784377e-07
  1.2

e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


GP Train RMSE: 37.757 ± 2.359
GP Test RMSE: 38.665 ± 2.529
GP R2: 0.9 ± 0.013
GP Time: 0.18 ± 0.02


In [15]:
# GPR with NN and FCM
gp_nn_tsl_train_accuracy = []
gp_nn_tsl_test_accuracy = []
gp_nn_tsl_r2s = []
nn_tsl_r2s = []
gp_nn_tsl_time = []
gpr_nn_tsl_model = None

# From NN and ts linear
for fold in range(number_of_folds):
    
    kernel = 1.0 * Matern(length_scale=1.0, nu=1.5)
    gpr_nn_tsl_model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9, normalize_y=False)
    tsl_model = get_tsl_model(x_train, y_train)
    nn_model

    start_time = time.time()
    inputs = tsl_model.cen
    ts_predictions = ts_relu_wrapper(tsl_model.predict(tsl_model.cen + 1e-8))
    nn_predictions = nn_model.predict(inputs)
    gpr_nn_tsl_model.fit(inputs, nn_predictions)
    time_used = time.time() - start_time

    train_means = gpr_nn_tsl_model.predict(x_train)
    test_means = gpr_nn_tsl_model.predict(x_test)

    train_rmse = root_mean_squared_error(y_train, train_means)
    test_rmse = root_mean_squared_error(y_test, test_means)

    gp_nn_tsl_train_accuracy.append(train_rmse)
    gp_nn_tsl_test_accuracy.append(test_rmse)
    gp_r2 = r2_score(y_test, test_means)
    gp_nn_tsl_r2s.append(gp_r2)
    gp_nn_tsl_time.append(time_used)
    nn_tsl_r2s.append(r2_score(ts_predictions, nn_predictions))

gp_train_accuracy_mean = np.round(np.mean(gp_nn_tsl_train_accuracy), 3)
gp_train_accuracy_std = np.round(np.std(gp_nn_tsl_train_accuracy), 3)
gp_test_accuracy_mean = np.round(np.mean(gp_nn_tsl_test_accuracy), 3)
gp_test_accuracy_std = np.round(np.std(gp_nn_tsl_test_accuracy),3)
gp_r2_mean = np.round(np.mean(gp_nn_tsl_r2s),3)
gp_r2_std = np.round(np.std(gp_nn_tsl_r2s),3)
gp_time_mean = np.round(np.mean(gp_nn_tsl_time),3)
gp_time_std = np.round(np.std(gp_nn_tsl_time),3)
nn_ts_r2_mean = np.round(np.mean(nn_tsl_r2s),3)
nn_ts_r2_std = np.round(np.std(nn_tsl_r2s),3)

print(f"GP Train RMSE: {gp_train_accuracy_mean} ± {gp_train_accuracy_std}")
print(f"GP Test RMSE: {gp_test_accuracy_mean} ± {gp_test_accuracy_std}")
print(f"GP R2: {gp_r2_mean} ± {gp_r2_std}")
print(f"GP Time: {gp_time_mean} ± {gp_time_std}")
print(f"TS NN R2: {nn_ts_r2_mean} ± {nn_ts_r2_std}")


FCM training RMSE: 15.919071599764736
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


FCM training RMSE: 15.918554580830921
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


FCM training RMSE: 16.065626610082866
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


...

e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


FCM training RMSE: 16.04694323475332
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


FCM training RMSE: 16.027818562166825
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
FCM training RMSE: 15.934658446652826
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step
GP Train RMSE: 37.573 ± 3.296
GP Test RMSE: 38.528 ± 3.456
GP R2: 0.901 ± 0.018
GP Time: 0.074 ± 0.008
TS NN R2: 0.998 ± 0.001


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
